# Imports


In [1]:
# For Data
import numpy as np
import pandas as pd
import pickle


from preprocessing import *
from feature_extractor import *
from model import *


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/robert/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /home/robert/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
test_file = 'Dataset/test.csv'
test_df = pd.read_csv(test_file)

In [3]:
# clean the data
test_data = cleanData(test_df, 'test' , clean=False, clearData=False)

# lemmatize the data
st = ISRIStemmer()
lemma = test_data.text.apply(lambda x: ([st.stem(word) for word in x.split()]))

# load the feature models
w2v_model = pickle.load(open('out/models/features/w2v_model.pkl', 'rb'))
bow_model = pickle.load(open('out/models/features/bow_model.pkl', 'rb'))
tfidf_model = pickle.load(open('out/models/features/tfidf_model.pkl', 'rb'))

# get the word embeddings for each tweet
x_test = get_features(w2v_model, bow_model, tfidf_model, lemma, 1, 1, 1) # get the word embeddings for each tweet
x_test= np.array([np.array(xi) for xi in x_test.to_numpy()])


# load the models
stance_model = loadModel('out/models/stance/clf.model')
category_model = loadModel('out/models/clf.model')

# get the predictions
stances = stance_model.predict(x_test)
categories = category_model.predict(x_test)

out_df = pd.DataFrame({'id': test_df.id, 'category': categories, 'stance': stances})

# save the predictions in csv file
out_df.to_csv('out/submission/predictions.csv', index=False)
